In [ ]:
import numpy as np
import h5py
import scipy.signal as scisignal

## Loading in Data

In [ ]:
with h5py.File('EP_preprocessed_x.h5', 'r') as hf:
    X = hf["EP_preprocessed_x_dataset"][:]

In [ ]:
print(X.shape)

## Reshaping X - optional

In [ ]:
def reshape_x(pre_x):
    new_x = np.reshape(pre_x, (pre_x.shape[0], pre_x.shape[1]*pre_x.shape[2]))
    return new_x

In [ ]:
#X = reshape_x(X)

## Spectral Filter Functions

In [ ]:
def lowhigh_spectral_filter(signalData, upperBoundary):
    b,a = scisignal.butter(4, [upperBoundary/125], btype='lowpass')
    filtered_signal_temp = scisignal.filtfilt(b,a,scisignal.detrend(signalData, type='constant'))
    
    b,a = scisignal.butter(4, [3/125], btype='highpass')
    filtered_signal_final = scisignal.filtfilt(b,a,scisignal.detrend(filtered_signal_temp, type='constant'))
    
    return filtered_signal_final

In [ ]:
def build_empty_array(signalData):
    x_spectral = np.empty((signalData.shape[0], signalData.shape[1], signalData.shape[2]))
    print(x_spectral.shape)
    return x_spectral

In [ ]:
def apply_spectral_filter(signalData, upperBoundary):
    x_spectral = build_empty_array(signalData)
    
    for event in range(len(signalData)):
        for channel in range(len(signalData[0])):
            x_spectral[event][channel] = lowhigh_spectral_filter(signalData[event][channel], upperBoundary)
    
    return x_spectral

In [ ]:
def save_data(x, dataName):
    with h5py.File(dataName + '_x.h5', 'w') as hf:
        hf.create_dataset(dataName + "_x_dataset", data=x)

## Hyper-Parameter Search

In [ ]:
# Hyperparameters
upper_boundaries = [20, 30, 50, 100]

In [ ]:
for upper_boundary in upper_boundaries:        
    # --- Spectral Filter ---
    X_spectral = apply_spectral_filter(X, upper_boundary)
    save_data(X_spectral, f"EP_spectral_{str(upper_boundary)}_upperBoundary")